<a href="https://colab.research.google.com/github/WKhisa/Machine-Learning--IPP/blob/main/Machine_Learning_IPP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

***Question***

> How to dentify the right employees for promotion




In [50]:
# Import pandas for data manipulation
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report, accuracy_score


In [70]:
#Get data
rawdataset_df = pd.read_csv('https://bit.ly/2ODZvLCHRDataset')
rawdataset_df.head()

,employee_id,department,region,education,gender,recruitment_channel,no_of_trainings,age,previous_year_rating,length_of_service,KPIs_met >80%,awards_won?,avg_training_score,is_promoted
0,65438,Sales & Marketing,region_7,Master's & above,f,sourcing,1,35,5.0,8,1,0,49,0
1,65141,Operations,region_22,Bachelor's,m,other,1,30,5.0,4,0,0,60,0
2,7513,Sales & Marketing,region_19,Bachelor's,m,sourcing,1,34,3.0,7,0,0,50,0
3,2542,Sales & Marketing,region_23,Bachelor's,m,other,2,39,1.0,10,0,0,50,0
4,48945,Technology,region_26,Bachelor's,m,other,1,45,3.0,2,0,0,73,0


**Exploring the dataset**

In [71]:
rawdataset_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54808 entries, 0 to 54807
Data columns (total 14 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   employee_id           54808 non-null  int64  
 1   department            54808 non-null  object 
 2   region                54808 non-null  object 
 3   education             52399 non-null  object 
 4   gender                54808 non-null  object 
 5   recruitment_channel   54808 non-null  object 
 6   no_of_trainings       54808 non-null  int64  
 7   age                   54808 non-null  int64  
 8   previous_year_rating  50684 non-null  float64
 9   length_of_service     54808 non-null  int64  
 10  KPIs_met >80%         54808 non-null  int64  
 11  awards_won?           54808 non-null  int64  
 12  avg_training_score    54808 non-null  int64  
 13  is_promoted           54808 non-null  int64  
dtypes: float64(1), int64(8), object(5)
memory usage: 5.9+ MB


**Standardize and cleaning the data**

In [72]:
rawdataset_df.columns

Index(['employee_id', 'department', 'region', 'education', 'gender',
       'recruitment_channel', 'no_of_trainings', 'age', 'previous_year_rating',
       'length_of_service', 'KPIs_met >80%', 'awards_won?',
       'avg_training_score', 'is_promoted'],
      dtype='object')

In [73]:
#standardize colum names
rawdataset_df.columns = rawdataset_df.columns.str.strip().str.lower().str.replace('>', 'gt_').str.replace('%', '_pct').str.replace(' ','_').str.replace('?', '')
rawdataset_df.columns

<ipython-input-73-d3929c074d12>:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  rawdataset_df.columns = rawdataset_df.columns.str.strip().str.lower().str.replace('>', 'gt_').str.replace('%', '_pct').str.replace(' ','_').str.replace('?', '')


Index(['employee_id', 'department', 'region', 'education', 'gender',
       'recruitment_channel', 'no_of_trainings', 'age', 'previous_year_rating',
       'length_of_service', 'kpis_met_gt_80_pct', 'awards_won',
       'avg_training_score', 'is_promoted'],
      dtype='object')

In [74]:
#drop unnecessary columns
rawdataset_df.drop(columns = ['employee_id','recruitment_channel','gender','region'], inplace = True)
rawdataset_df.head()

,department,education,no_of_trainings,age,previous_year_rating,length_of_service,kpis_met_gt_80_pct,awards_won,avg_training_score,is_promoted
0,Sales & Marketing,Master's & above,1,35,5.0,8,1,0,49,0
1,Operations,Bachelor's,1,30,5.0,4,0,0,60,0
2,Sales & Marketing,Bachelor's,1,34,3.0,7,0,0,50,0
3,Sales & Marketing,Bachelor's,2,39,1.0,10,0,0,50,0
4,Technology,Bachelor's,1,45,3.0,2,0,0,73,0


In [75]:
#check missing data
rawdataset_df.isna().sum()

department                 0
education               2409
no_of_trainings            0
age                        0
previous_year_rating    4124
length_of_service          0
kpis_met_gt_80_pct         0
awards_won                 0
avg_training_score         0
is_promoted                0
dtype: int64

In [76]:
#check duplicates
rawdataset_df.duplicated().sum()

7245

In [77]:
#drop missing rows
rawdataset_df.dropna(inplace = True)
rawdataset_df.isna().sum()

department              0
education               0
no_of_trainings         0
age                     0
previous_year_rating    0
length_of_service       0
kpis_met_gt_80_pct      0
awards_won              0
avg_training_score      0
is_promoted             0
dtype: int64

In [61]:
rawdataset_df.shape

(48660, 10)

In [78]:
# One-hot encode categorical variables
encoded_df = pd.get_dummies(rawdataset_df, columns=['department', 'education'], drop_first=True)
encoded_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 48660 entries, 0 to 54807
Data columns (total 18 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   no_of_trainings               48660 non-null  int64  
 1   age                           48660 non-null  int64  
 2   previous_year_rating          48660 non-null  float64
 3   length_of_service             48660 non-null  int64  
 4   kpis_met_gt_80_pct            48660 non-null  int64  
 5   awards_won                    48660 non-null  int64  
 6   avg_training_score            48660 non-null  int64  
 7   is_promoted                   48660 non-null  int64  
 8   department_Finance            48660 non-null  uint8  
 9   department_HR                 48660 non-null  uint8  
 10  department_Legal              48660 non-null  uint8  
 11  department_Operations         48660 non-null  uint8  
 12  department_Procurement        48660 non-null  uint8  
 13  d

**Data Modelling**

In [79]:
#split data to features and target
features = encoded_df.drop(columns = ['is_promoted'], axis =1)
target = encoded_df['is_promoted']

In [80]:
# training and testing sets
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

# Initialize and train a DecisionTreeClassifier model
model = DecisionTreeClassifier(random_state=42)
model.fit(X_train, y_train)


DecisionTreeClassifier(random_state=42)

In [81]:
# Make predictions on the test set
y_pred = model.predict(X_test)

# Evaluate the model's performance
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print(accuracy)
print(report)


0.895396629675298
              precision    recall  f1-score   support

           0       0.94      0.94      0.94      8865
           1       0.42      0.44      0.43       867

    accuracy                           0.90      9732
   macro avg       0.68      0.69      0.69      9732
weighted avg       0.90      0.90      0.90      9732

